# BP

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from dfply import *

In [2]:
import psycopg2

conn = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password="June3060!!",
    database = "mimic_data",
    port="5432"
)

cursor = conn.cursor()
print("database connected")

database connected


In [3]:
df = pd.read_csv("../Data/reintubation_df_240127.csv", index_col="Unnamed: 0")

In [4]:
df2 = df.copy()

In [5]:
df2

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,8416.0,2126-05-05 18:20:00,4430.0
9004,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,4269.0,NaN,NaN
9005,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,4201.0,NaN,NaN
9006,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,1155.0,NaN,NaN


----

In [6]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_icu.d_items;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
d_items = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)

경과 시간 :  0.03229188919067383


In [7]:
d_items

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,None,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,None,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4009,230077,Documented on DC Summary (PICC),Documented on DC Summary (PICC),chartevents,PICC Line Insertion,None,Text,NaN,NaN
4010,230078,Documented on DC Summary (Thora),Documented on DC Summary (Thora),chartevents,Thoracentesis,None,Text,NaN,NaN
4011,230083,Replacement Fluid (Pre-Filter),Replacement Fluid (Pre-Filter),chartevents,Dialysis,None,Text,NaN,NaN
4012,230084,Replacement Fluid (Post-Filter),Replacement Fluid (Post-Filter),chartevents,Dialysis,None,Text,NaN,NaN


## d_items에서 데이터 확인

In [8]:
d_items[d_items['label'].str.contains('ressure')]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
6,220050,Arterial Blood Pressure systolic,ABPs,chartevents,Routine Vital Signs,mmHg,Numeric,90.0,140.0
7,220051,Arterial Blood Pressure diastolic,ABPd,chartevents,Routine Vital Signs,mmHg,Numeric,60.0,90.0
8,220052,Arterial Blood Pressure mean,ABPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
9,220056,Arterial Blood Pressure Alarm - Low,ABP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN
10,220058,Arterial Blood Pressure Alarm - High,ABP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3946,229891,Pressure Ulcer #8- Date Indentified,Pressure Ulcer #8- Date Indentified,datetimeevents,Skin - Impairment,None,Date and time,NaN,NaN
3947,229892,Pressure Ulcer #9- Date Indentified,Pressure Ulcer #9- Date Indentified,datetimeevents,Skin - Impairment,None,Date and time,NaN,NaN
3948,229893,Pressure Ulcer #10- Date Indentified,Pressure Ulcer #10- Date Indentified,datetimeevents,Skin - Impairment,None,Date and time,NaN,NaN
3954,229899,Left Ventricular Pressure Signal - Systolic,Left Ventricular Pressure Signal - Systolic,chartevents,Impella,mmHg,Numeric,NaN,NaN


## d_items에서 데이터 2차 확인

* 220050, 220051, 220052, 220179, 220180, 220181, 224167, 224643, 227242, 227243
* systolic 수축 혈압 -> 220050, 220179, 224167, 227243
* diastolic 이완 혈압 -> 220051, 220180, 224643, 227242
* mean -> 220052, 220181
    * mean = 1/3 * systolic + 2/3 * diastolic

In [9]:
d_items[d_items['label'].str.contains('Blood Pressure')] 

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
6,220050,Arterial Blood Pressure systolic,ABPs,chartevents,Routine Vital Signs,mmHg,Numeric,90.0,140.0
7,220051,Arterial Blood Pressure diastolic,ABPd,chartevents,Routine Vital Signs,mmHg,Numeric,60.0,90.0
8,220052,Arterial Blood Pressure mean,ABPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
9,220056,Arterial Blood Pressure Alarm - Low,ABP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN
10,220058,Arterial Blood Pressure Alarm - High,ABP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
24,220179,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
25,220180,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
26,220181,Non Invasive Blood Pressure mean,NBPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
332,223751,Non-Invasive Blood Pressure Alarm - High,NBP Alarm - High,chartevents,Alarms,mmHg,Numeric,NaN,NaN
333,223752,Non-Invasive Blood Pressure Alarm - Low,NBP Alarm - Low,chartevents,Alarms,mmHg,Numeric,NaN,NaN


In [10]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_icu.chartevents where itemid in (220050, 220051, 220052, 220179, 220180, 220181, 224167, 224643, 227242, 227243);")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
bp_chart = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)

경과 시간 :  155.11332058906555


In [105]:
bp_chart.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032,29079034,39553978,47007,2180-07-23 21:01:00,2180-07-23 22:15:00,220179,82,82.0,mmHg,0
1,10000032,29079034,39553978,47007,2180-07-23 21:01:00,2180-07-23 22:15:00,220180,59,59.0,mmHg,0
2,10000032,29079034,39553978,47007,2180-07-23 21:01:00,2180-07-23 22:15:00,220181,63,63.0,mmHg,0
3,10000032,29079034,39553978,47007,2180-07-23 22:00:00,2180-07-23 22:15:00,220179,85,85.0,mmHg,0
4,10000032,29079034,39553978,47007,2180-07-23 22:00:00,2180-07-23 22:15:00,220180,55,55.0,mmHg,0


## bp_chart

* columns => subject_id, hadm_id, stay_id, charttime, storetime, itemid, valuenum 추출

In [93]:
cols = ["subject_id", "hadm_id", "stay_id", "charttime", "storetime","itemid", "valuenum"]

In [94]:
bp_chart2 = bp_chart[cols].copy()
bp_chart2

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,valuenum
0,10000032,29079034,39553978,2180-07-23 21:01:00,2180-07-23 22:15:00,220179,82.0
1,10000032,29079034,39553978,2180-07-23 21:01:00,2180-07-23 22:15:00,220180,59.0
2,10000032,29079034,39553978,2180-07-23 21:01:00,2180-07-23 22:15:00,220181,63.0
3,10000032,29079034,39553978,2180-07-23 22:00:00,2180-07-23 22:15:00,220179,85.0
4,10000032,29079034,39553978,2180-07-23 22:00:00,2180-07-23 22:15:00,220180,55.0
...,...,...,...,...,...,...,...
19002623,19999987,23865745,36195440,2145-11-02 23:40:00,2145-11-02 23:43:00,220180,70.0
19002624,19999987,23865745,36195440,2145-11-02 23:40:00,2145-11-02 23:43:00,220181,81.0
19002625,19999987,23865745,36195440,2145-11-03 05:00:00,2145-11-03 05:05:00,220179,113.0
19002626,19999987,23865745,36195440,2145-11-03 05:00:00,2145-11-03 05:05:00,220180,73.0


In [95]:
bp_chart2["itemid"].value_counts()

itemid
220179    4067219
220180    4066337
220181    4064537
220052    2272013
220050    2264001
220051    2263647
224167       1400
224643       1296
227243       1136
227242       1042
Name: count, dtype: int64

In [96]:
bp_systolic = bp_chart2[bp_chart2["itemid"].isin([220050, 220179, 224167, 227243])]
bp_systolic = bp_systolic.copy()
bp_systolic.rename(columns={"valuenum":"systolic"}, inplace=True)

In [97]:
bp_systolic.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,systolic
0,10000032,29079034,39553978,2180-07-23 21:01:00,2180-07-23 22:15:00,220179,82.0
3,10000032,29079034,39553978,2180-07-23 22:00:00,2180-07-23 22:15:00,220179,85.0
6,10000032,29079034,39553978,2180-07-23 19:00:00,2180-07-23 19:59:00,220179,93.0
9,10000032,29079034,39553978,2180-07-23 20:00:00,2180-07-23 21:01:00,220179,90.0
12,10000032,29079034,39553978,2180-07-23 14:11:00,2180-07-23 14:17:00,220179,84.0


In [98]:
bp_systolic.shape

(6333756, 7)

In [99]:
bp_diastolic = bp_chart2[bp_chart2["itemid"].isin([220051, 220180, 224643, 227242])]
bp_diastolic = bp_diastolic.copy()
bp_diastolic.rename(columns={"valuenum":"diastolic"}, inplace=True)

In [100]:
bp_diastolic.shape

(6332322, 7)

In [101]:
bp_diastolic.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,diastolic
1,10000032,29079034,39553978,2180-07-23 21:01:00,2180-07-23 22:15:00,220180,59.0
4,10000032,29079034,39553978,2180-07-23 22:00:00,2180-07-23 22:15:00,220180,55.0
7,10000032,29079034,39553978,2180-07-23 19:00:00,2180-07-23 19:59:00,220180,41.0
10,10000032,29079034,39553978,2180-07-23 20:00:00,2180-07-23 21:01:00,220180,57.0
13,10000032,29079034,39553978,2180-07-23 14:11:00,2180-07-23 14:17:00,220180,48.0


In [102]:
bp_mean = bp_chart2[bp_chart2["itemid"].isin([220052, 220181])]
bp_mean=bp_mean.copy()
bp_mean.rename(columns={"valuenum":"bp_mean"}, inplace=True)

In [103]:
bp_mean.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,bp_mean
2,10000032,29079034,39553978,2180-07-23 21:01:00,2180-07-23 22:15:00,220181,63.0
5,10000032,29079034,39553978,2180-07-23 22:00:00,2180-07-23 22:15:00,220181,62.0
8,10000032,29079034,39553978,2180-07-23 19:00:00,2180-07-23 19:59:00,220181,56.0
11,10000032,29079034,39553978,2180-07-23 20:00:00,2180-07-23 21:01:00,220181,64.0
14,10000032,29079034,39553978,2180-07-23 14:11:00,2180-07-23 14:17:00,220181,56.0


In [139]:
bp_data = bp_systolic >> left_join(bp_diastolic, by=("subject_id", "hadm_id", "stay_id", "charttime", "storetime"))
bp_data1 = bp_data >> left_join(bp_mean, by=("subject_id", "hadm_id", "stay_id", "charttime", "storetime"))
bp_data2 = bp_data1 >> select("subject_id", "hadm_id", "stay_id", "charttime", "storetime", "systolic", "diastolic", "bp_mean")
bp_data2

,subject_id,hadm_id,stay_id,charttime,storetime,systolic,diastolic,bp_mean
0,10000032,29079034,39553978,2180-07-23 21:01:00,2180-07-23 22:15:00,82.0,59.0,63.0
1,10000032,29079034,39553978,2180-07-23 22:00:00,2180-07-23 22:15:00,85.0,55.0,62.0
2,10000032,29079034,39553978,2180-07-23 19:00:00,2180-07-23 19:59:00,93.0,41.0,56.0
3,10000032,29079034,39553978,2180-07-23 20:00:00,2180-07-23 21:01:00,90.0,57.0,64.0
4,10000032,29079034,39553978,2180-07-23 14:11:00,2180-07-23 14:17:00,84.0,48.0,56.0
...,...,...,...,...,...,...,...,...
7050116,19999987,23865745,36195440,2145-11-03 15:00:00,2145-11-03 15:42:00,111.0,68.0,78.0
7050117,19999987,23865745,36195440,2145-11-03 16:00:00,2145-11-03 16:35:00,93.0,57.0,66.0
7050118,19999987,23865745,36195440,2145-11-04 16:53:00,2145-11-04 16:55:00,152.0,92.0,102.0
7050119,19999987,23865745,36195440,2145-11-02 23:40:00,2145-11-02 23:43:00,115.0,70.0,81.0


In [140]:
bp_data2.isna().sum()

subject_id         0
hadm_id            0
stay_id            0
charttime          0
storetime          0
systolic           0
diastolic      15299
bp_mean       159670
dtype: int64

## bp_data 데이터 전처리

* diastolic, bp_mean 두 값이 모두 null 값인 데이터 제거

In [141]:
bp_data2[(bp_data2["diastolic"].isna()) & (bp_data2["bp_mean"].isna())]

,subject_id,hadm_id,stay_id,charttime,storetime,systolic,diastolic,bp_mean
169,10001725,25563031,31205490,2110-04-12 17:00:00,2110-04-12 17:04:00,112.0,NaN,NaN
853,10009686,29681222,34139812,2164-04-30 07:37:00,2164-04-30 07:54:00,142.0,NaN,NaN
1009,10010058,26359957,33060379,2147-11-18 11:00:00,2147-11-18 11:30:00,118.0,NaN,NaN
1859,10004401,29988601,32773003,2144-01-27 05:00:00,2144-01-27 05:02:00,82.0,NaN,NaN
2594,10011365,26712576,37153661,2166-02-14 16:01:00,2166-02-14 16:07:00,160.0,NaN,NaN
...,...,...,...,...,...,...,...,...
7047397,19997367,20617667,35616526,2126-05-09 03:02:00,2126-05-09 04:08:00,107.0,NaN,NaN
7047583,19997367,20617667,35616526,2126-04-23 21:00:00,2126-04-23 21:08:00,99.0,NaN,NaN
7047872,19997473,27787494,32134105,2173-09-19 10:16:00,2173-09-19 10:52:00,90.0,NaN,NaN
7047994,19997473,27787494,32568753,2173-09-13 23:01:00,2173-09-13 23:01:00,127.0,NaN,NaN


In [142]:
bp_data2.drop(index=bp_data2[(bp_data2["diastolic"].isna()) & (bp_data2["bp_mean"].isna())].index, inplace=True)

* bp_mean이 null 값이라면 ?
* diastolic + (systolic - diastolic) / 3 = mean

In [143]:
bp_data2[bp_data2["bp_mean"].isna()].index

Index([     31,     123,     158,     168,     208,     219,     224,     228,
           235,     241,
       ...
       7049606, 7049675, 7049682, 7049877, 7049897, 7049997, 7050039, 7050042,
       7050048, 7050049],
      dtype='int64', length=147887)

In [144]:
bp_data2.loc[bp_data2[bp_data2["bp_mean"].isna()].index, "bp_mean"] = bp_data2["diastolic"] + (bp_data2["systolic"] - bp_data2["diastolic"]) / 3

* diastolic이 null 이라면?
* 3d + sys - 1d = 3m
    * 2d + sys = 3m
    * 2d = 3m - sys
    * d = (3m-sys) / 2

In [153]:
bp_data2[bp_data2["diastolic"].isna()]

,subject_id,hadm_id,stay_id,charttime,storetime,systolic,diastolic,bp_mean
1460,10002155,23822395,33685454,2129-08-08 18:00:00,2129-08-08 17:59:00,121.0,NaN,74.0
2083,10010867,22429197,39880770,2148-01-06 05:00:00,2148-01-06 05:15:00,120.0,NaN,80.0
4739,10002428,28662225,38875437,2156-04-25 17:00:00,2156-04-25 17:37:00,101.0,NaN,85.0
5851,10002495,24982426,36753294,2141-05-25 04:00:00,2141-05-25 04:42:00,97.0,NaN,81.0
9887,10014354,27487226,38017367,2148-07-09 22:01:00,2148-07-10 00:22:00,187.0,NaN,161.0
...,...,...,...,...,...,...,...,...
7044326,19994505,23109063,35505750,2185-11-03 05:00:00,2185-11-03 05:19:00,93.0,NaN,58.0
7045509,19995780,21942461,36805359,2125-10-23 17:00:00,2125-10-23 17:11:00,128.0,NaN,79.0
7047434,19997367,20617667,35616526,2126-04-22 14:00:00,2126-04-22 14:52:00,104.0,NaN,65.0
7048096,19997473,27787494,32134105,2173-09-20 04:02:00,2173-09-20 04:23:00,98.0,NaN,57.0


In [154]:
bp_data2.loc[bp_data2[bp_data2["diastolic"].isna()].index, "diastolic"] = ((3*bp_data2["bp_mean"]) - bp_data2["systolic"]) / 2

In [160]:
bp_data2.isna().sum()

subject_id    0
hadm_id       0
stay_id       0
charttime     0
storetime     0
systolic      0
diastolic     0
bp_mean       0
dtype: int64

## bp 데이터 합치기

In [199]:
'''
* 기존 데이터엔 int_time이 NaN이라면, int_stayid도 NaN이다 (ext_time 동일)

* 기본적으로 혈압을 재는 건 본격적인 수술 전, 중, 후에 재는 것으로 1차적 생각을 진행
* 그렇다면 1차적으로 삽관 시술 전 혈압을 잴 것이고, 발관 이후에 혈압을 잴 것이다. (중간은 데이터를 넣기가 애매하므로 제외)
* 그러므로 우선순위는 삽관 시술 전 혈압 데이터이며, 만약 그 데이터가 없다면 발관 이후 혈압 데이터를 넣는다.

* bp_data의 charttime이 기록된 시간, storetime이 데이터가 저장된 시간이므로 혈압을 잰 시간은 charttime으로 지정
* 혈압 같은 경우 int_stayid(int_time)를 기준으로 일단 charttime 대입. 
    * charttime <= int_time
* int_time 없다면 ext_stayid(ext_time)을 기준으로 charttime 대입.
    * charttime >= ext_time
* 위의 기준은 변경이 될 수 있음.
'''

def group_df_bp(group):
    # group의 index, row를 하나씩 확인
    for idx, row in group.iterrows():

        # 만약, int_stayid가 null 값이 아니라면
        if pd.notnull(row["int_stayid"]):

            # bp_data의 subject_id, stay_id가 row의 subject_id, int_stayid를 만족하는 데이터 프레임을 만든다.
            bp_data_df = bp_data2[(bp_data2["subject_id"] == row["subject_id"]) & (bp_data2["hadm_id"] == row["hadm_id"]) & (bp_data2["stay_id"] == row["int_stayid"])]

            # row의 intubationtime보다 작거나 같은 bp_data_df charttime 중 가장 마지막 index를 반환
            suitable_location = bp_data_df[bp_data_df["charttime"] <= row["intubationtime"]].last_valid_index()
            try:
                # 반환된 index를 이용하여 값 할당
                group.loc[idx, ["systolic","diastolic", "bp_mean"]] = bp_data_df.loc[suitable_location, ["systolic", "diastolic", "bp_mean"]].values
            except:
                # 만약 반환된 index가 없다면 (row의 intubationtime이 가장 최소 값이라면)
                try:
                    # bp_data의 charttime이 가장 작은 값을 index로 가지는 BP 값 할당
                    group.loc[idx, ["systolic","diastolic", "bp_mean"]] = bp_data_df.loc[suitable_location, ["systolic", "diastolic", "bp_mean"]].values
                except:
                    # 그래도 값이 없다면 nan 값 할당
                    group.loc[idx, ["systolic","diastolic", "bp_mean"]] = np.nan
        # 만약 int_stayid가 null 값이라면                    
        else:

            # gcs의 subject_id, stay_id가 row의 subject_id, ext_stayid를 만족하는 데이터 프레임을 만든다.
            bp_data_df = bp_data2[(bp_data2["subject_id"] == row["subject_id"]) & (bp_data2["hadm_id"] == row["hadm_id"]) & (bp_data2["stay_id"] == row["ext_stayid"])]

            # row의 extubationtime보다 크거나 같은 bp_data2 의 charttime 중 가장 최근 index를 반환
            suitable_location = bp_data_df[bp_data_df["charttime"] >= row["extubationtime"]].first_valid_index()
            try:
                # 반환된 index를 이용하여 값 할당
                group.loc[idx, ["systolic","diastolic", "bp_mean"]] = bp_data_df.loc[suitable_location, ["systolic", "diastolic", "bp_mean"]].values
            except:
                # 만약 반환된 index가 없다면 (extubationtime이 최대 값이라면)
                try:
                    # gcs_df의 charttime이 가장 큰 값을 index로 가지는 BP 값 할당
                    group.loc[idx, ["systolic","diastolic", "bp_mean"]] = bp_data_df.loc[suitable_location, ["systolic", "diastolic", "bp_mean"]].values
                except:
                    # 그래도 값이 없다면 nan 값 할당
                    group.loc[idx, ["systolic","diastolic", "bp_mean"]] = np.nan
    # group return
    return group

In [162]:
df2.head()

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN


In [164]:
df_bp = df2.groupby(["subject_id", "hadm_id"], group_keys=False).apply(group_df_bp)
df_bp

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime,systolic,diastolic,bp_mean
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,...,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN,180.0,12.0,46.0
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,...,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN,95.0,65.0,71.0
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,...,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN,115.0,54.0,68.0
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,...,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0,133.0,45.0,67.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,...,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN,133.0,45.0,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,...,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,8416.0,2126-05-05 18:20:00,4430.0,91.0,75.0,78.0
9004,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,...,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,4269.0,NaN,NaN,91.0,75.0,78.0
9005,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,...,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,4201.0,NaN,NaN,111.0,74.0,82.0
9006,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,...,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,1155.0,NaN,NaN,170.0,85.0,113.0


In [166]:
df_bp.head()

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime,systolic,diastolic,bp_mean
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,...,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN,180.0,12.0,46.0
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,...,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN,95.0,65.0,71.0
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,...,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN,115.0,54.0,68.0
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,...,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0,133.0,45.0,67.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,...,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN,133.0,45.0,67.0


In [197]:
df_bp[df_bp["ext_stayid"].isna()]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime,systolic,diastolic,bp_mean
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,...,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN,115.0,54.0,68.000000
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,...,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN,133.0,45.0,67.000000
6,10007818,22987108,32359580.0,2146-06-10 16:37:00,2146-06-22 12:49:00,224385.0,86.2,NaN,NaN,NaN,...,NaN,2146-07-12 00:00:00,2146-07-12 20:50:00,NaN,NaN,NaN,NaN,63.0,41.0,48.333333
7,10010058,26359957,33060379.0,2147-11-18 02:34:00,2147-11-18 02:35:00,224385.0,60.0,NaN,NaN,NaN,...,NaN,2147-11-19 04:00:00,2147-11-19 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10017531,22580355,35526828.0,2159-09-22 19:30:00,2159-09-23 16:01:00,224385.0,119.0,NaN,NaN,NaN,...,NaN,2159-10-24 13:40:00,NaN,NaN,NaN,NaN,NaN,164.0,107.0,118.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8984,19971771,26230047,38668070.0,2117-04-26 00:21:00,2117-04-26 00:22:00,224385.0,90.0,NaN,NaN,NaN,...,NaN,2117-05-25 11:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8989,19979469,29434840,37363141.0,2202-02-22 18:26:00,2202-02-22 21:40:00,224385.0,86.0,NaN,NaN,NaN,...,NaN,2202-02-28 18:20:00,NaN,NaN,NaN,NaN,NaN,133.0,71.0,87.000000
8991,19983257,21588174,31169972.0,2166-06-13 14:21:00,2166-06-19 09:20:00,224385.0,118.6,NaN,NaN,NaN,...,NaN,2166-06-19 21:15:00,2166-06-19 21:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8998,19990427,29695607,34058245.0,2182-01-04 00:00:00,2182-01-14 04:21:00,224385.0,100.0,NaN,NaN,NaN,...,NaN,2182-01-24 00:00:00,2182-01-24 23:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
bp_data2[(bp_data2["subject_id"] == 10020740) & (bp_data2["hadm_id"] == 23831430) & (bp_data2["stay_id"] == 35044342.0)]

,subject_id,hadm_id,stay_id,charttime,storetime,systolic,diastolic,bp_mean
15812,10020740,23831430,35044342,2150-03-20 17:00:00,2150-03-20 18:19:00,115.0,44.0,63.000000
15813,10020740,23831430,35044342,2150-03-20 18:00:00,2150-03-20 18:20:00,122.0,94.0,101.000000
15814,10020740,23831430,35044342,2150-03-21 15:00:00,2150-03-21 16:12:00,133.0,72.0,88.000000
15815,10020740,23831430,35044342,2150-03-21 16:00:00,2150-03-21 16:12:00,145.0,82.0,97.000000
15816,10020740,23831430,35044342,2150-03-23 07:00:00,2150-03-23 07:16:00,139.0,69.0,87.000000
...,...,...,...,...,...,...,...,...
15948,10020740,23831430,35044342,2150-03-22 11:00:00,2150-03-22 12:20:00,137.0,74.0,90.000000
15949,10020740,23831430,35044342,2150-03-22 12:00:00,2150-03-22 12:20:00,106.0,58.0,71.000000
15950,10020740,23831430,35044342,2150-03-22 16:00:00,2150-03-22 16:40:00,152.0,60.0,90.666667
15951,10020740,23831430,35044342,2150-03-22 17:00:00,2150-03-22 17:20:00,154.0,87.0,101.000000


In [175]:
df_bp[(df_bp["systolic"].isna()) | (df_bp["diastolic"].isna()) | (df_bp["bp_mean"].isna())]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime,systolic,diastolic,bp_mean
7,10010058,26359957,33060379.0,2147-11-18 02:34:00,2147-11-18 02:35:00,224385.0,60.0,NaN,NaN,NaN,...,NaN,2147-11-19 04:00:00,2147-11-19 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,10021487,28998349,38197705.0,2116-12-03 00:23:00,2116-12-03 00:25:00,224385.0,124.0,NaN,NaN,NaN,...,NaN,2116-12-28 13:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,10032409,24735796,35929854.0,2133-01-16 17:40:00,2133-01-16 21:11:00,224385.0,70.0,NaN,NaN,NaN,...,NaN,2133-01-21 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,10054464,22926852,33613175.0,2122-06-11 13:04:00,2122-06-11 13:30:00,224385.0,74.0,33613175.0,2122-06-11 17:35:00,227194.0,...,Planned Extubation,2122-06-11 21:20:00,2122-06-11 21:20:00,NaN,245.0,NaN,NaN,NaN,NaN,NaN
55,10090454,29525590,38669202.0,2169-02-05 17:14:00,2169-02-06 19:00:00,224385.0,54.0,NaN,NaN,NaN,...,NaN,2169-02-11 02:14:00,2169-02-11 02:14:00,NaN,NaN,2169-02-10 16:22:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8939,19917446,20856545,36858538.0,2123-11-26 18:48:00,2124-01-11 10:00:00,224385.0,56.0,NaN,NaN,NaN,...,NaN,2124-01-21 16:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8980,19970265,24514478,38356273.0,2115-03-28 17:37:00,2115-03-28 18:00:00,224385.0,67.9,NaN,NaN,NaN,...,NaN,2115-04-18 14:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8984,19971771,26230047,38668070.0,2117-04-26 00:21:00,2117-04-26 00:22:00,224385.0,90.0,NaN,NaN,NaN,...,NaN,2117-05-25 11:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8991,19983257,21588174,31169972.0,2166-06-13 14:21:00,2166-06-19 09:20:00,224385.0,118.6,NaN,NaN,NaN,...,NaN,2166-06-19 21:15:00,2166-06-19 21:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


-----

# 내용 정리
* d_items에서 주요한 blood pressure itemid(label) 추출
* 해당 event가 있는 chartevents에서 해당 itemid만 추출하여 확인
* 수축기, 이완기, 평균 값 데이터프레임을 따로 추출, left join
    * 혈압 데이터 특성을 확인 했을 시, subject_id, hadm_id, time이 동일해야함.
* 이완기, 평균 값 중 두개 모두 NaN 값인 데이터 drop.
* 둘 중 하나만 NaN 값이라면, 평균 혈압 공식 (diastolic + (systolic - diastolic) / 3 = mean) 이용해서 구함
* 함수를 이용하여 본연의 paired된 데이터 프레임에 대입

## 질문 사항
1. 혈압 데이터 NaN 값을 채우는 것에 대한 의문.
    * 이완기 및 평균 값 둘중 하나 NaN 값일 시, 평균 공식 (이완기 + (수축기-이완기) / 3)으로 NaN 값을 채운 상황


2. BP의 시간대 기준을을 잡는 것에 대한 의문.
    * 혈압 같은 경우 int_stayid(int_time)를 기준으로 일단 charttime 대입. 
        * charttime <= int_time (삽관 전 시간 중 가장 최근 시간)
    * int_time 없다면 ext_stayid(ext_time)을 기준으로 charttime 대입.
        * charttime >= ext_time (발관 후 시간 중 가장 최근 시간)
    * bp의 시간대 기준을 이렇게 잡았는데 BP의 기준을 이렇게 잡아도 괜찮을지 의문
